In [158]:
from os import listdir, makedirs, stat
from os.path import isfile, join, exists, splitext
import subprocess
import math
import filecmp

In [81]:
def files_in_dir(directory: str):
    return sorted([f for f in listdir(directory) if isfile(join(directory, f))])

In [132]:
jpeg30_file_names = [("jpeg30/" + f) for f in files_in_dir("jpeg30/")]

In [133]:
print(*jpeg30_file_names)

jpeg30/airplane30.jpg jpeg30/arctichare30.jpg jpeg30/baboon30.jpg jpeg30/cat30.jpg jpeg30/fruits30.jpg jpeg30/frymire30.jpg jpeg30/girl30.jpg jpeg30/lena30.jpg jpeg30/monarch30.jpg jpeg30/peppers30.jpg jpeg30/pool30.jpg jpeg30/sails30.jpg jpeg30/serrano30.jpg jpeg30/tulips30.jpg jpeg30/watch30.jpg


In [134]:
jpeg80_file_names = [("jpeg80/" + f) for f in files_in_dir("jpeg80/")]

In [135]:
print(*jpeg80_file_names)

jpeg80/airplane80.jpg jpeg80/arctichare80.jpg jpeg80/baboon80.jpg jpeg80/cat80.jpg jpeg80/fruits80.jpg jpeg80/frymire80.jpg jpeg80/girl80.jpg jpeg80/lena80.jpg jpeg80/monarch80.jpg jpeg80/peppers80.jpg jpeg80/pool80.jpg jpeg80/sails80.jpg jpeg80/serrano80.jpg jpeg80/tulips80.jpg jpeg80/watch80.jpg


In [136]:
def test_encode(executable: str, file_in: str, file_out: str, quality: int, ignoreOutput = True):
    _stdout = subprocess.DEVNULL if ignoreOutput else None
    subprocess.run(executable + " {} {} {}".format(file_in, file_out, quality), shell=True, check=True, stdout=_stdout)

In [137]:
def test_encode_all(executable: str, files_in, out_dir: str, quality: int, ignoreOutput = True):
    if not exists(out_dir):
        makedirs(out_dir)
    for file_in in files_in:
        filename = splitext(file_in)[0].split('/')[-1]
        file_out = out_dir + "/" + filename
        test_encode(executable, file_in, file_out, quality, ignoreOutput)

In [138]:
test_encode_all("./jpeg-recompress", jpeg30_file_names, "encoded30", 30)

In [139]:
test_encode_all("./jpeg-recompress", jpeg80_file_names, "encoded80", 80)

In [201]:
def report(encodedDir: str, origDir: str):
    compressed_filenames = sorted([f for f in listdir(encodedDir) if isfile(join(encodedDir, f))])
    totalCompressedSize = 0
    totalOrigSize = 0
    for filename in compressed_filenames:
        origSize = stat(origDir + filename + ".jpg").st_size
        totalOrigSize += origSize
        size = stat(encodedDir + filename).st_size
        totalCompressedSize += size
        change = (size - origSize) * 100 / origSize
        changeStr = ("+" if change > 0 else "") + str(round(change, 2))
        print("{:12s} & {:8d} & {:8d} & {}".format(filename, origSize, size, changeStr))
    print(totalOrigSize)
    print(totalCompressedSize)
    print(round((totalCompressedSize - totalOrigSize) * 100 / totalOrigSize, 2))

In [202]:
report("encoded30/", "jpeg30/")

airplane30   &    19207 &    18282 & -4.82
arctichare30 &    12080 &    10928 & -9.54
baboon30     &    36113 &    34871 & -3.44
cat30        &    35902 &    35222 & -1.89
fruits30     &    17646 &    17346 & -1.7
frymire30    &   200860 &   191932 & -4.44
girl30       &    24310 &    22487 & -7.5
lena30       &    17578 &    17116 & -2.63
monarch30    &    28501 &    27285 & -4.27
peppers30    &    18746 &    18663 & -0.44
pool30       &     7467 &     6249 & -16.31
sails30      &    45512 &    43270 & -4.93
serrano30    &    58288 &    57129 & -1.99
tulips30     &    37972 &    37200 & -2.03
watch30      &    46741 &    41973 & -10.2
606923
579953
-4.44


In [203]:
report("encoded80/", "jpeg80/")

airplane80   &    44079 &    44848 & +1.74
arctichare80 &    26569 &    26725 & +0.59
baboon80     &    88465 &    87456 & -1.14
cat80        &    81974 &    82540 & +0.69
fruits80     &    45303 &    46420 & +2.47
frymire80    &   440857 &   419128 & -4.93
girl80       &    59979 &    60514 & +0.89
lena80       &    43872 &    44959 & +2.48
monarch80    &    64055 &    64725 & +1.05
peppers80    &    47929 &    48809 & +1.84
pool80       &    14492 &    14426 & -0.46
sails80      &   105830 &   104622 & -1.14
serrano80    &   138167 &   136352 & -1.31
tulips80     &    85764 &    85907 & +0.17
watch80      &   101074 &    99738 & -1.32
1388409
1367169
-1.53


In [173]:
compressed_filenames = sorted([f for f in listdir("encoded80") if isfile(join("encoded80", f))])
retSz = 0
for filename in compressed_filenames:
    origSize = stat("jpeg80/" + filename + ".jpg").st_size
    size = stat("encoded80/" + filename).st_size
    retSz += min([origSize, size])
print(retSz)

1361246


In [177]:
compressed_filenames = sorted([f for f in listdir("encoded80switching") if isfile(join("encoded80switching", f))])
retSz = 0
for filename in compressed_filenames:
    #origSize = stat("jpeg80/" + filename + ".jpg").st_size
    size = stat("encoded80switching/" + filename).st_size
    retSz += size
print(retSz)

1361261


In [150]:
def test_decode(executable: str, file_in: str, file_out: str, ignoreOutput = True):
    _stdout = subprocess.DEVNULL if ignoreOutput else None
    subprocess.run(executable + " {} {}".format(file_in, file_out), shell=True, check=True, stdout=_stdout)

In [151]:
def test_decode_all(executable: str, files_in, out_dir: str, ignoreOutput = True):
    if not exists(out_dir):
        makedirs(out_dir)
    for file_in in files_in:
        filename = splitext(file_in)[0].split('/')[-1]
        file_out = out_dir + "/" + filename + ".jpg"
        test_decode(executable, file_in, file_out, ignoreOutput)

In [152]:
encoded30_file_names = [("encoded30/" + f) for f in files_in_dir("encoded30")]

In [153]:
print(*encoded30_file_names)

encoded30/airplane30 encoded30/arctichare30 encoded30/baboon30 encoded30/cat30 encoded30/fruits30 encoded30/frymire30 encoded30/girl30 encoded30/lena30 encoded30/monarch30 encoded30/peppers30 encoded30/pool30 encoded30/sails30 encoded30/serrano30 encoded30/tulips30 encoded30/watch30


In [154]:
test_decode_all("./jpeg-recompress-back", encoded30_file_names, "decoded30")

In [155]:
encoded80_file_names = [("encoded80/" + f) for f in files_in_dir("encoded80")]

In [156]:
test_decode_all("./jpeg-recompress-back", encoded80_file_names, "decoded80")

In [160]:
filecmp.dircmp("jpeg30/", "decoded30/").report()

diff jpeg30/ decoded30/
Identical files : ['airplane30.jpg', 'arctichare30.jpg', 'baboon30.jpg', 'cat30.jpg', 'fruits30.jpg', 'frymire30.jpg', 'girl30.jpg', 'lena30.jpg', 'monarch30.jpg', 'peppers30.jpg', 'pool30.jpg', 'sails30.jpg', 'serrano30.jpg', 'tulips30.jpg', 'watch30.jpg']


In [161]:
filecmp.dircmp("jpeg80/", "decoded80/").report()

diff jpeg80/ decoded80/
Identical files : ['airplane80.jpg', 'arctichare80.jpg', 'baboon80.jpg', 'cat80.jpg', 'fruits80.jpg', 'frymire80.jpg', 'girl80.jpg', 'lena80.jpg', 'monarch80.jpg', 'peppers80.jpg', 'pool80.jpg', 'sails80.jpg', 'serrano80.jpg', 'tulips80.jpg', 'watch80.jpg']


In [164]:
round((584049 - 611019) / 611019 * 100, 3)

-4.414

In [165]:
round((1371265 - 1392505) / 1392505 * 100, 3)

-1.525

In [166]:
test_encode_all("./jpeg-recompress-switching", jpeg30_file_names, "encoded30switching", 30)

In [179]:
test_encode_all("./jpeg-recompress-switching", jpeg80_file_names, "encoded80switching", 80)

In [198]:
report("encoded30switching/", "jpeg30/")

airplane30   &    19207 &    18283 & -4.81
arctichare30 &    12080 &    10929 & -9.53
baboon30     &    36113 &    34872 & -3.44
cat30        &    35902 &    35223 & -1.89
fruits30     &    17646 &    17347 & -1.69
frymire30    &   200860 &   191933 & -4.44
girl30       &    24310 &    22488 & -7.49
lena30       &    17578 &    17117 & -2.62
monarch30    &    28501 &    27286 & -4.26
peppers30    &    18746 &    18664 & -0.44
pool30       &     7467 &     6250 & -16.3
sails30      &    45512 &    43271 & -4.92
serrano30    &    58288 &    57130 & -1.99
tulips30     &    37972 &    37201 & -2.03
watch30      &    46741 &    41974 & -10.2
579968
-4.44


In [205]:
report("encoded80switching/", "jpeg80/")

airplane80   &    44079 &    44080 & +0.0
arctichare80 &    26569 &    26570 & +0.0
baboon80     &    88465 &    87457 & -1.14
cat80        &    81974 &    81975 & +0.0
fruits80     &    45303 &    45304 & +0.0
frymire80    &   440857 &   419129 & -4.93
girl80       &    59979 &    59980 & +0.0
lena80       &    43872 &    43873 & +0.0
monarch80    &    64055 &    64056 & +0.0
peppers80    &    47929 &    47930 & +0.0
pool80       &    14492 &    14427 & -0.45
sails80      &   105830 &   104623 & -1.14
serrano80    &   138167 &   136353 & -1.31
tulips80     &    85764 &    85765 & +0.0
watch80      &   101074 &    99739 & -1.32
1388409
1361261
-1.96


In [200]:
(1392505 - 1361261) / 1392505

0.022437262343761782

In [170]:
round((584064 - 611019) / 611019 * 100, 3)

-4.411

In [211]:
round((1388409 - 1361261 + 110 * 6) / 1388409 * 100, 3)

2.003